In [ ]:
%%javascript
$('#header').toggle()

In [ ]:
import ipywidgets as widgets
import json
import pandas as pd
import qgrid
import requests
from ipyleaflet import FullScreenControl, LayerGroup, Map, Marker
from IPython.display import display, HTML, IFrame
from pyvis.network import Network

In [ ]:
debug = False

In [ ]:
layout_main = {
    'width': '100%',
    'height': '100%',
}
layout_dropdown = {
    'width': '150px',
}
layout_button = {
    'width': '50px',
}
layout_tree = {
    'width': '100%',
    'height': '500px',
}
layout_table = {
    'width': '3000px',
    'height': '100%',
}

In [ ]:
image_header = widgets.Image(
    value=open('images/DI_NLP_Header.png', 'rb').read(),
    format='png',
    layout=layout_main,
)

In [ ]:
input_url = widgets.Text(
    placeholder='TextCat URL',
    layout=layout_main,
)
input_text = widgets.Textarea(
    placeholder='Text for analysis',
    layout=layout_main,
)

In [ ]:
dropdown_set_sentedges = widgets.Dropdown(
    value=0,
    options=[
        ('Sentence edges on', 0),
        ('Sentence edges off', 1),
    ],
    layout=layout_dropdown,
)
dropdown_set_corefs = widgets.Dropdown(
    value=0,
    options=[
        ('All corefs', 0),
        ('Simple corefs', 1),
        ('Neural corefs', 2),
        ('No corefs', 3),
    ],
    layout=layout_dropdown,
)
dropdown_set_output = widgets.Dropdown(
    value=0,
    options=[
        ('All outputs', 0),
        ('Text', 1),
        ('Tree', 2),
        ('Graph', 3),
        ('Map', 4),
        ('Table', 5),
    ],
    layout=layout_dropdown,
)
button_toggle_output_debug = widgets.Button(
    icon='bug',
    tooltip='Toggle debug mode',
    layout=layout_button,
)
button_toggle_output_open = widgets.Button(
    icon='list',
    tooltip='Toggle all output displays',
    layout=layout_button,
)
button_clear = widgets.Button(
    icon='refresh',
    tooltip='Clear input and output',
    layout=layout_button,
)
button_go = widgets.Button(
    icon='play',
    tooltip='Go!',
    layout=layout_button,
)

In [ ]:
output_message = widgets.Output()
output_text_1 = widgets.Output()
output_text_2 = widgets.Output()
output_tree_1 = widgets.Output(
    layout=layout_tree,
)
output_tree_2 = widgets.Output(
    layout=layout_tree,
)
output_graph_1 = widgets.Output()
output_graph_2 = widgets.Output()
output_graph_3 = widgets.Output()
output_map = widgets.Output()
output_table = widgets.Output(
    layout=layout_table,
)

In [ ]:
accordion_text_1 = widgets.Accordion(
    children=[output_text_1],
    layout=layout_main,
)
accordion_text_2 = widgets.Accordion(
    children=[output_text_2],
    layout=layout_main,
)
accordion_tree_1 = widgets.Accordion(
    children=[output_tree_1],
    layout=layout_main,
)
accordion_tree_2 = widgets.Accordion(
    children=[output_tree_2],
    layout=layout_main,
)
accordion_graph_1 = widgets.Accordion(
    children=[output_graph_1],
    layout=layout_main,
)
accordion_graph_2 = widgets.Accordion(
    children=[output_graph_2],
    layout=layout_main,
)
accordion_graph_3 = widgets.Accordion(
    children=[output_graph_3],
    layout=layout_main,
)
accordion_map = widgets.Accordion(
    children=[output_map],
    layout=layout_main,
)
accordion_table = widgets.Accordion(
    children=[output_table],
    layout=layout_main,
)

accordion_text_1.set_title(0, 'Text: Raw')
accordion_text_2.set_title(0, 'Text' + (': Re-node' if debug else ''))
accordion_tree_1.set_title(0, 'Tree: Raw')
accordion_tree_2.set_title(0, 'Tree' + (': Re-node' if debug else ''))
accordion_graph_1.set_title(0, 'Graph: Raw')
accordion_graph_2.set_title(0, 'Graph: Re-node')
accordion_graph_3.set_title(0, 'Graph' + (': Re-node Re-edge' if debug else ''))
accordion_map.set_title(0, 'Map')
accordion_table.set_title(0, 'Table')

In [ ]:
children_header = [
    image_header,
    input_url,
    input_text,
]
children_control = [
    dropdown_set_sentedges,
    dropdown_set_corefs,
    dropdown_set_output,
    button_toggle_output_debug,
    button_toggle_output_open,
    button_clear,
    button_go,
    output_message,
]
children_outputs = {
    False: {
        0: {
            'children_output': [
                accordion_text_2,
                accordion_tree_2,
                accordion_graph_3,
                accordion_map,
                accordion_table,
            ],
            'outputs': [
                output_text_2,
                output_tree_2,
                output_graph_3,
                output_map,
                output_table,
            ],
        },
        1: {
            'children_output': [accordion_text_2],
            'outputs': [output_text_2],
        },
        2: {
            'children_output': [accordion_tree_2],
            'outputs': [output_tree_2],
        },
        3: {
            'children_output': [accordion_graph_3],
            'outputs': [output_graph_3],
        },
        4: {
            'children_output': [accordion_map],
            'outputs': [output_map],
        },
        5: {
            'children_output': [accordion_table],
            'outputs': [output_table],
        },
    },
    True: {
        0: {
            'children_output': [
                accordion_text_1,
                accordion_text_2,
                accordion_tree_1,
                accordion_tree_2,
                accordion_graph_1,
                accordion_graph_2,
                accordion_graph_3,
                accordion_map,
                accordion_table,
            ],
            'outputs': [
                output_text_1,
                output_text_2,
                output_tree_1,
                output_tree_2,
                output_graph_1,
                output_graph_2,
                output_graph_3,
                output_map,
                output_table,
            ],
        },
        1: {
            'children_output': [accordion_text_1, accordion_text_2],
            'outputs': [output_text_1, output_text_2],
        },
        2: {
            'children_output': [accordion_tree_1, accordion_tree_2],
            'outputs': [output_tree_1, output_tree_2],
        },
        3: {
            'children_output': [accordion_graph_1, accordion_graph_2, accordion_graph_3],
            'outputs': [output_graph_1, output_graph_2, output_graph_3],
        },
        4: {
            'children_output': [accordion_map],
            'outputs': [output_map],
        },
        5: {
            'children_output': [accordion_table],
            'outputs': [output_table],
        },
    },
}
children_output = []
outputs = []

HBox_control = widgets.HBox(
    children=children_control,
)
VBox_main = widgets.VBox(
    layout = {
        'width': '90%',
        'left': '10%',
        'padding': '5px',
#         'border': '1px solid black',
    },
)

In [ ]:
# g_nx is not currently used for anything. Started working on it here to use NetworkX graph methods if needed:
#     g_nx = nx.DiGraph()
#     g_nx.add_nodes_from([(token['idx_tok'], token) for token in tokens])
#     g_nx.add_edges_from([(token['idx_tok_head'], token['idx_tok'], {'dep': token['dep']}) for token in tokens])

#     g_pyvis.enable_physics(True)
#     g_pyvis.barnes_hut()
#     g_pyvis.show_buttons(filter_=['nodes'])
#     g_pyvis.show_buttons(filter_=['edges'])
#     g_pyvis.show_buttons(filter_=['physics'])

# Note that this does not copy over node and edge attributes, so it's a bit useless really and hence we just rebuild the graph from scratch with pyvis below:
#     g_pyvis.from_nx(g_nx)
#     g_pyvis.add_nodes(
#         [ent['ent'] for ent in ents],
#         color=[ent['color'] for ent in ents],
#         title=[
#             pd.DataFrame(
#                 [(key[1], ent[key[0]])
#                  for key in keys_display_order
#                  if key[0] in ent.keys()],
#             ).style.set_table_styles([
#                 {'selector': 'th.col_heading',
#                  'props': [('display', 'none')]},
#                 {'selector': 'th.row_heading',
#                  'props': [('display', 'none')]},
#             ]).render()
#             for ent in ents
#         ],
#     )

@output_message.capture(clear_output=True, wait=True)
def set_output_message(message):
    display(HTML(message))

@output_text_1.capture(clear_output=True, wait=True)
def set_output_text_1(html_ents):
    accordion_text_1.selected_index = 0
    display(HTML(html_ents))

@output_text_2.capture(clear_output=True, wait=True)
def set_output_text_2(html_ents):
    accordion_text_2.selected_index = 0
    display(HTML(html_ents))

@output_tree_1.capture(clear_output=True, wait=True)
def set_output_tree_1(html_dep):
    accordion_tree_1.selected_index = 0
    display(HTML(html_dep))

@output_tree_2.capture(clear_output=True, wait=True)
def set_output_tree_2(html_dep):
    accordion_tree_2.selected_index = 0
    display(HTML(html_dep))

@output_graph_1.capture(clear_output=True, wait=True)
def set_output_graph_1(g_pyvis):
    g_pyvis.show('images/DI_NLP_Graph_raw.html')
    accordion_graph_1.selected_index = 0
    display(
        IFrame(
            'images/DI_NLP_Graph_raw.html',
            width='100%',
            height='1000px',
        ),
    )

@output_graph_2.capture(clear_output=True, wait=True)
def set_output_graph_2(g_pyvis):
    g_pyvis.show('images/DI_NLP_Graph_renode.html')
    accordion_graph_2.selected_index = 0
    display(
        IFrame(
            'images/DI_NLP_Graph_renode.html',
            width='100%',
            height='1000px',
        ),
    )

@output_graph_3.capture(clear_output=True, wait=True)
def set_output_graph_3(g_pyvis):
    g_pyvis.show('images/DI_NLP_Graph_renode_reedge.html')
    accordion_graph_3.selected_index = 0
    display(
        IFrame(
            'images/DI_NLP_Graph_renode_reedge.html',
            width='100%',
            height='1000px',
        ),
    )

def set_output_graph(nodes, edges, mode):
    g_pyvis = Network(
        directed=True,
        notebook=True,
        width='100%',
        height='100%',
    )
    g_pyvis.prep_notebook()
    g_pyvis.force_atlas_2based(
        gravity=-100,
        central_gravity=0.005,
        spring_length=50,
        spring_strength=0.1,
        damping=2.0,
        overlap=0.5,
    )
    for node in nodes:
        label = \
              str(node['idx_tok']) \
            + ': ' \
            + str(node['idx_chr']) \
            + ': ' \
            + node['pos'] \
            + ': ' \
            + (
                (node['ent_type_plus_detail1'] + ': ')
                if (node['ent_type'] not in {None, 'di-event'})
                else ''
              ) \
            + node['text']
        g_pyvis.add_node(
            node['idx_tok'],
            label=label,
            title=label,
            font='14px gillsans #424242',
            color=node['color'],
        )
    for edge in edges:
        g_pyvis.add_edge(
            edge['idx_tok_from'],
            edge['idx_tok_to'],
            label=edge['di_dep'],
            title=edge['di_dep'],
            font='12px gillsans-italic #A9A9A9',
        )
    if   (mode == 1):
        set_output_graph_1(g_pyvis)
    elif (mode == 2):
        set_output_graph_2(g_pyvis)
    elif (mode == 3):
        set_output_graph_3(g_pyvis)

@output_map.capture(clear_output=True, wait=True)
def set_output_map(ents):
    m = Map(
        center=(20.0, 0.0),
        zoom=2,
    )
    markers = []
    for ent in ents.keys():
        if  (ents[ent]['ent_type'] == 'LOC') \
        and ('geo_lat' in ents[ent].keys()) \
        and ('geo_lon' in ents[ent].keys()):
            markers.append(Marker(
                location=(ents[ent]['geo_lat'], ents[ent]['geo_lon']),
                title=ent,
                draggable=False,
            ))
    m.add_layer(LayerGroup(
        layers=tuple(markers),
    ))
#     m.add_layer(MarkerCluster(
#         markers=tuple(markers),
#     ))
    m.add_control(FullScreenControl())
    accordion_map.selected_index = 0
    display(m)

@output_table.capture(clear_output=True, wait=True)
def set_output_table(ents, keys_display_order):
    ents = [ents[ent] for ent in ents.keys()]
    ents = sorted(ents, key=lambda ent:ent['tokens_idx'][0])
    df = pd.DataFrame(
        [[ent[key[0]] if key[0] in ent.keys() else ''
          for key in keys_display_order]
         for ent in ents],
        columns=[key[1] for key in keys_display_order],
    )
    qgrid_widget = qgrid.show_grid(
        df,
        grid_options={
            'enableColumnReorder': True,
        },
#         show_toolbar=True, # This gives a full-screen button, but also add/remove row buttons which we don't want ...
    )
    accordion_table.selected_index = 0
    display(qgrid_widget)
#     df = df.style.set_table_styles([
#         {'selector': 'td',
#          'props': [('text-align', 'left')]},
#         {'selector': 'th.col_heading',
#          'props': [('text-align', 'left')]},
#         {'selector': 'th.row_heading',
#          'props': [('text-align', 'right')]},
#     ])
#     display(df)
#     display(TableDisplay(df))

In [ ]:
def set_output(dummy):
    global children_output
    global outputs
    children_output = children_outputs[debug][dropdown_set_output.value]['children_output']
    outputs = children_outputs[debug][dropdown_set_output.value]['outputs']
    VBox_main.children = children_header + [HBox_control] + children_output

def clear_output(dummy):
    for child in children_outputs[True][0]['children_output']:
        child.selected_index = None
    for output in children_outputs[True][0]['outputs']:
        output.clear_output()

def clear_input_output(dummy):
    input_text.value=''
    output_message.clear_output()
    clear_output(None)

def toggle_control(dummy):
    control_enabled = not children_control[0].disabled
    for child in children_control:
        child.disabled = True if control_enabled else False

def toggle_output_debug(dummy):
    global debug
    if (not debug):
        debug = True
        button_toggle_output_debug.button_style = 'warning'
        accordion_text_2.set_title(0, 'Text: Re-node')
        accordion_tree_2.set_title(0, 'Tree: Re-node')
        accordion_graph_3.set_title(0, 'Graph: Re-node Re-edge')
    else:
        debug = False
        button_toggle_output_debug.button_style = ''
        accordion_text_2.set_title(0, 'Text')
        accordion_tree_2.set_title(0, 'Tree')
        accordion_graph_3.set_title(0, 'Graph')
    set_output(None)

def toggle_output_open(dummy):
    output_open = True in [(child.selected_index == 0) for child in children_output]
    for child in children_output:
        child.selected_index = None if output_open else 0

def go(dummy):
    if   (not input_url.value) and (not input_text.value):
        set_output_message('ERROR: Enter TextCat URL and text for analysis')
    elif (not input_url.value):
        set_output_message('ERROR: Enter TextCat URL')
    elif (not input_text.value):
        set_output_message('ERROR: Enter text for analysis')
    else:
        toggle_control(None)
        set_output_message('Processing ...') # + '<i class="fa fa-gear fa-spin" style="font-size:18px"></i>')
# TODO: Explicitly specify as unicode input
        r = requests.post(
            input_url.value,
            data=json.dumps({'text': input_text.value}),
            params={
                'return_app': 'true',
                'return_app_debug': 'true' if debug else 'false',
                'return_sentedges': str((dropdown_set_output.value in [0,3]) and (dropdown_set_sentedges.value == 0)).lower(),
                'return_corefs_simple': str((dropdown_set_output.value in [0,3]) and (dropdown_set_corefs.value in [0,1])).lower(),
                'return_corefs_neural': str((dropdown_set_output.value in [0,3]) and (dropdown_set_corefs.value in [0,2])).lower(),
            },
            headers={
                'Content-Type': 'application/json'
            },
        )
        toggle_control(None)
        clear_output(None)
        if r.ok and r.json():
            set_output_message('Done in: ' + str(r.json()['timer']) + ' s')
            if  (dropdown_set_output.value in [0,5]) \
            and ('ents' in r.json().keys()) \
            and ('keys_display_order' in r.json().keys()):
                set_output_table(r.json()['ents'], r.json()['keys_display_order'])
            if  (dropdown_set_output.value in [0,4]) \
            and ('ents' in r.json().keys()):
                set_output_map(r.json()['ents'])
            if  (dropdown_set_output.value in [0,3]) \
            and ('nodes_renode_reedge' in r.json().keys()) \
            and ('edges_renode_reedge' in r.json().keys()):
                set_output_graph(r.json()['nodes_renode_reedge'], r.json()['edges_renode_reedge'], 3)
            if  (debug) \
            and (dropdown_set_output.value in [0,3]) \
            and ('nodes_renode' in r.json().keys()) \
            and ('edges_renode' in r.json().keys()):
                set_output_graph(r.json()['nodes_renode'], r.json()['edges_renode'], 2)
            if  (debug) \
            and (dropdown_set_output.value in [0,3]) \
            and ('nodes_raw' in r.json().keys()) \
            and ('edges_raw' in r.json().keys()):
                set_output_graph(r.json()['nodes_raw'], r.json()['edges_raw'], 1)
            if  (dropdown_set_output.value in [0,2]) \
            and ('html_dep_renode' in r.json().keys()) \
            and (r.json()['html_dep_renode']):
                set_output_tree_2(r.json()['html_dep_renode'])
            if  (debug) \
            and (dropdown_set_output.value in [0,2]) \
            and ('html_dep_raw' in r.json().keys()) \
            and (r.json()['html_dep_raw']):
                set_output_tree_1(r.json()['html_dep_raw'])
            if  (dropdown_set_output.value in [0,1]) \
            and ('html_ents_renode' in r.json().keys()):
                set_output_text_2(r.json()['html_ents_renode'])
            if  (debug) \
            and (dropdown_set_output.value in [0,1]) \
            and ('html_ents_raw' in r.json().keys()):
                set_output_text_1(r.json()['html_ents_raw'])
        else:
            set_output_message('ERROR: No data received')
            clear_output(None)

In [ ]:
dropdown_set_output.observe(set_output)
button_toggle_output_debug.on_click(toggle_output_debug)
button_toggle_output_open.on_click(toggle_output_open)
button_clear.on_click(clear_input_output)
button_go.on_click(go)

In [ ]:
set_output(None)
clear_output(None)
display(VBox_main)